In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("EL Nino (5-D).csv")         # Importing El- nino dataset
df.head()

,Observation,Year,Month,Day,Date,Latitude,Longitude,Zonal Winds,Meridional Winds,Humidity,Air Temp,Sea Surface Temp
0,4060,93,5,9,930509,-0.02,-109.96,-2.1,2.1,81.2,26.80,27.02
1,4061,93,5,10,930510,-0.02,-109.96,-3.4,1.4,84.2,26.95,26.91
2,4062,93,5,11,930511,-0.02,-109.96,-3.8,2.2,84.9,26.98,26.78
3,4063,93,5,12,930512,-0.02,-109.96,-3.0,1.5,86.9,26.93,26.74
4,4064,93,5,13,930513,-0.02,-109.96,-4.5,1.9,87.6,27.01,26.82


In [ ]:
data = df.drop(df.columns[0:7],axis=1)            # Removing spatio-temporial attributes
data.head()

,Zonal Winds,Meridional Winds,Humidity,Air Temp,Sea Surface Temp
0,-2.1,2.1,81.2,26.80,27.02
1,-3.4,1.4,84.2,26.95,26.91
2,-3.8,2.2,84.9,26.98,26.78
3,-3.0,1.5,86.9,26.93,26.74
4,-4.5,1.9,87.6,27.01,26.82


In [ ]:
data

,Zonal Winds,Meridional Winds,Humidity,Air Temp,Sea Surface Temp
0,-2.1,2.1,81.2,26.80,27.02
1,-3.4,1.4,84.2,26.95,26.91
2,-3.8,2.2,84.9,26.98,26.78
3,-3.0,1.5,86.9,26.93,26.74
4,-4.5,1.9,87.6,27.01,26.82
...,...,...,...,...,...
93930,-6.8,-5.3,81.3,27.52,28.17
93931,-5.1,-0.4,94.1,26.04,28.14
93932,-4.3,-3.3,93.2,25.80,27.87
93933,-6.1,-4.8,81.3,27.17,27.93


In [ ]:
#Shuffling the dataset row-vise
shuffled_data = data.sample(frac = 1, random_state = 42)      # frac = 1 means you want to sample the entire DataFrame

# Reset the index if you want to reset the index after shuffling
shuffled_data.reset_index(drop=True, inplace=True)

# Total dataset S for later use
S = shuffled_data.copy()

shuffled_data.head()

s1 = shuffled_data.sample(frac=0.000596, random_state=42)
s1_complement = shuffled_data.drop(s1.index)

s2 = s1_complement.sample(frac=0.000596, random_state=42)
s1_s2_complement = s1_complement.drop(s2.index)

sprime=s1_s2_complement.sample(frac=0.000596,random_state=42)

s1.reset_index(drop = True, inplace = True)
s2.reset_index(drop = True, inplace = True)
sprime.reset_index(drop = True, inplace = True)

print(s1.shape)
print(s2.shape)
print(sprime.shape)

len_s1 = s1.shape[0]
len_s2 = s2.shape[0]
len_sprime = sprime.shape[0]

s1_matrix = s1.to_numpy()
s2_matrix = s2.to_numpy()
sprime_matrix=sprime.to_numpy()

(56, 5)
(56, 5)
(56, 5)


$\Huge{\text{Required Functions for Algorithm}}$

# $ \textbf{Equation 5:} \\
\left\{\begin{array}{l}
p\left(x_{j} \mid \omega_{i}\right)=\frac{1}{(2 \pi)^{k / 2}\left|\Sigma_{x_{i}}\right|^{1 / 2}} \exp \left(-\frac{1}{2}\left(x_{j}-x_{i}\right)^{T} \Sigma_{x_{i}}^{-1}\left(x_{j}-x_{i}\right)\right) \\
p\left(x_{i} \mid \omega_{i}\right)=0, \quad i, j=1,2, \ldots\left|S_{1}\right|, i \neq j \\
\quad i=1,2, \ldots\left|S_{1}\right|
\end{array}\right.
$

In [ ]:
k = 5        # Dimensionality of s1

def small_p(xi, xj, cov):

    cov_1 = cov

    det_cov = np.linalg.det(cov)
    epsilon = 1e-2
    det_cov = max(det_cov, epsilon)

    if det_cov == 0.01:
        alpha = 1e-2
        cov_1 = cov_1 + alpha * np.identity(5)

    a = 1 / (((2 * np.pi) ** (k / 2)) * det_cov ** 0.5)
    b = -0.5 * ((xj - xi).T @ np.linalg.inv(cov_1) @ (xj - xi))

    if b > 1000:
        print('b value', b)
    b = min(b, 50)
    p = a * np.exp(b)

    return p

# $ \textbf{Equation 4:} \\
\begin{cases}P\left(\omega_{i} \mid x_{j}\right)=\frac{p\left(x_{j} \mid \omega_{i}\right)}{\sum_{t=1}^{\left|S_{1}\right|} p\left(x_{j} \mid \omega_{t}\right)} & i, j=1,2, \ldots\left|S_{1}\right|, i \neq j \\ P\left(\omega_{i} \mid x_{i}\right)=0 & i=1,2, \ldots\left|S_{1}\right|\end{cases}
$

In [ ]:
def capital_p(i, j, cov_mat):
    summation = 0
    xi = s1_matrix[i, :].reshape(-1, 1)
    xj = s1_matrix[j, :].reshape(-1, 1)

    for k in range(len_s1):
        if k != j:
            xk = s1_matrix[k, :].reshape(-1, 1)
            summation += small_p(xk,xj, cov_mat[k])

    # Avoid division by zero
    summation = np.clip(summation,1e-4,np.inf)
    P = small_p(xi,xj,cov_mat[i]) / summation

    return  P

# $ \textbf{Equation 3:} \\
\Sigma_{x_{i}}=\frac{\sum_{x_{j} \in S_{1}} P\left(\omega_{i} \mid x_{j}\right)\left(x_{j}-x_{i}\right)\left(x_{j}-x_{i}\right)^{T}}{\sum_{x_{j} \in S_{1}} P\left(\omega_{i} \mid x_{j}\right)}
$

In [ ]:
def cov_xi(i,cov_mat):
    denominator = 0
    numerator = 0

    xi = s1_matrix[i, :].reshape(-1, 1)
    for j in range(len_s1):
        if i!=j:
            xj = s1_matrix[j, :].reshape(-1, 1)

            P = capital_p(i, j, cov_mat)

            denominator += P
            numerator += P * ((xj - xi) @ (xj - xi).T)

    return numerator / denominator

In [ ]:
def L(cov_mat):
    L_new = 0
    for j in range(len_s1):
        xj = s1_matrix[j, :].reshape(-1, 1)
        summation = 0
        for i in range(len_s1):
            if i!=j:
                xi = s1_matrix[i, :].reshape(-1, 1)
                summation+=small_p(xi,xj,cov_mat[i])

        summation=summation/(len_s1-1)
        L_new+=np.log(summation)

    return L_new[0][0]

# $ \textbf{Equation 2:}$
$ \ L=\sum_{x_{j} \in S_{1}} \log \left[\sum_{x_{i} \in S_{1} \wedge i \neq j} \frac{1}{\left|S_{1}\right|-1} G\left(\Sigma_{x_{i}}, x_{j}-x_{i}\right)\right]
$

$\Huge{\text{Algorithm 1}}$

In [ ]:
# Initial guess

cov_mat = []

for i in range(len_s1):
    xi = s1_matrix[i, :].reshape(-1, 1)
    mat = 0
    for j in range(len_s1):
        xj = s1_matrix[j, :].reshape(-1, 1)
        mat += (xj-xi) @ (xj-xi).T

    cov_mat.append(mat/(len_s1-1))

In [ ]:
t = 0
phi = 0.01
L_prev = -250

while t < 100:

    temp = []

    for i in range(len_s1):
        temp.append(cov_xi(i,cov_mat))

    cov_mat = temp.copy()

    for i in range(len_s1):
        if np.linalg.det(cov_mat[i]) <= 0:
            alpha = 1e-2
            cov_mat[i] = cov_mat[i] + alpha * np.identity(5)

    L_new = L(cov_mat)

    print('L_prev=',L_prev)
    print('L_new=',L_new)
    print('t=',t)
    print('-'*50)

    if abs((L_new-L_prev)/L_prev) < phi:
        break

    else:
        L_prev=L_new
        t+=1

L_prev= -250
L_new= -599.1623016577415
t= 0
--------------------------------------------------
L_prev= -599.1623016577415
L_new= -579.3466089904226
t= 1
--------------------------------------------------
L_prev= -579.3466089904226
L_new= -552.9270483785061
t= 2
--------------------------------------------------
L_prev= -552.9270483785061
L_new= -512.9229743436242
t= 3
--------------------------------------------------
L_prev= -512.9229743436242
L_new= -460.45778011123275
t= 4
--------------------------------------------------
L_prev= -460.45778011123275
L_new= -415.12455318612183
t= 5
--------------------------------------------------
L_prev= -415.12455318612183
L_new= -393.6483069484127
t= 6
--------------------------------------------------
L_prev= -393.6483069484127
L_new= -382.66824462109906
t= 7
--------------------------------------------------
L_prev= -382.66824462109906
L_new= -379.86632270412395
t= 8
--------------------------------------------------


$\Huge{\text{Algorithm 2}}$

In [ ]:
def F(T):
    s = 0
    for i in range(len_s1):
        xi = s1_matrix[i, :].reshape(-1, 1)
        s += small_p(xi,T,cov_mat[i])
    f = np.log(s/len_s1)
    return f

def EstVar(estsize,beta):
    Est = []
    for t in range(1,estsize):
        R = s2.sample(n=len(s2), replace=True, random_state=42)
        R = R.to_numpy()
        F_R = []
        for i in range(len_s2):
            T = R[i, :].reshape(-1, 1)
            F_R.append(F(T))

        var_F_R = np.var(np.array(F_R))
        Est.append((len_s2/(len_s2-1))*var_F_R)

    V =  np.percentile(Est, estsize*(1 - beta))
    var_delta = (len_sprime + ((len_sprime**2) / len_s2))*V
    return var_delta

$\Huge{\text{Algorithm 3}}$

In [ ]:
from scipy.stats import norm

def critical_value(p,stepsize):
    estsize = 100
    M = int(np.ceil((p/stepsize)-1))
    C = []
    for i in range(1, M+1):
        alpha_i = i*stepsize
        beta_i = p-alpha_i
        var_delta = EstVar(estsize,beta_i)

        mean = 0

        std_dev = np.sqrt(var_delta)
        c = norm.ppf(alpha_i, loc = mean, scale = std_dev)
        C.append(c)

    C_max = max(C)

    return C_max

# $\textbf{Equation 7:}$
 $
\begin{aligned}
\delta= & \operatorname{LLH}\left(\mathscr{K}_{S_{1}}, S^{\prime}\right)-\frac{\left|S^{\prime}\right|}{\left|S_{2}\right|} \times \operatorname{LLH}\left(\mathscr{K}_{S_{1}}, S_{2}\right) \\
= & \log \left\{\prod_{y \in S^{\prime}} \mathscr{K}_{S_{1}}(y)\right\}-\frac{\left|S^{\prime}\right|}{\left|S_{2}\right|} \times \log \left\{\prod_{y \in S_{2}} \mathscr{K}_{S_{1}}(y)\right\} \\
= & \sum_{y \in S^{\prime}} \log \sum_{x \in S_{1}} \frac{1}{\left|S_{1}\right|} G\left(\Sigma_{x}, y-x\right) -\sum_{y \in S_{2}} \frac{\left|S^{\prime}\right|}{\left|S_{2}\right|} \times \log \left\{\sum_{x \in S_{1}} \frac{1}{\left|S_{1}\right|} G\left(\Sigma_{x}, y-x\right)\right\}
\end{aligned}
$

In [ ]:
def delta():

    a = 0

    for j in range(len_sprime):
        y = sprime_matrix[j, :].reshape(-1, 1)
        summation = 0

        for i in range(len_s1):
            x = s1_matrix[i, :].reshape(-1, 1)
            summation += small_p(x,y,cov_mat[i])

        summation=np.clip(summation,1e-4,np.inf)
        a += np.log(summation/len_s1)

    b = 0

    for j in range(len_s2):
        y = s2_matrix[j, :].reshape(-1, 1)
        summation = 0

        for i in range(len_s1):
            x = s1_matrix[i, :].reshape(-1, 1)
            summation += small_p(x,y,cov_mat[i])

        b += (np.log(summation/len_s1))*len_sprime/len_s2

    delta = b-a

    return delta[0][0]

$\Huge{\text{Experiment 2}}$

$\text{We have run the test for 10 instances for the 3 noise types(gaussian 5-d, add 1-d, scale 1-d) because of the high time complexity.}$
$\text{Still we are getting expected results}$

$\Large{\text{Full Dimensional Changes:}}$

$\text{We have S1, S2 and S' are samples from same distributions: FS.}$
$\text{S' is intentionally made different from FS to simulate a scenario where there's been a distributional change.}$

$\text{To simulating distributional change involves creating S' as a mixture of the original distribution FS}$
$\text{and another distribution FC}$

$\text{S' is obtained by sampling from the original data set and adding Gaussian noise in some points to all dimensions.}$
$\text{This essentially perturbs the original data with Gaussian noise.}$

In [ ]:
means = np.mean(sprime_matrix , axis = 0)
stds = np.std(sprime_matrix , axis = 0)

In [ ]:
# generate sprime matrix 10 instances

fn = 0

for inst in range(10):
    print('instance:',inst+1,'/10')
    sprime_instance = data.sample(frac = 0.000596, replace = True, random_state = 42)
    sprime_matrix = sprime_instance.to_numpy()

    noise_matrix = np.zeros((12,5))   # We take noise

    for i in range(12):
        for j in range(5):
            noise_matrix[i,j] = np.random.normal(loc = means[j] , scale = stds[j])

    index = np.random.randint(0, 56, size = 12)

    for i in range(12):
        sprime_matrix[index[i]] = noise_matrix[i]          #Adding gaussian noise

    p = 0.08
    stepsize = 0.002

    Delta = delta()

    C_max = critical_value(p, stepsize)

    if Delta < C_max:
        print('THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S ')

    else:
        fn += 1
        print('THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S ')

    print('fn:',fn)
    print('-'*80)

print('False negative rate:', fn/10)

instance: 1 /10
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
--------------------------------------------------------------------------------
instance: 2 /10
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
--------------------------------------------------------------------------------
instance: 3 /10
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
--------------------------------------------------------------------------------
instance: 4 /10
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
--------------------------------------------------------------------------------
instance: 5 /10
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
--------------------------------------------------------------------------------
instance: 6 /10
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
f

$\text{Expected result: 0.}$

$\text{Obtained result: 0.}$

$\Large{\text{Single Dimensional Changes:}}$

$\text{S' is created by adding a standard 1-D Gaussian variable to a single randomly-chosen dimension of the original data set.}$

$\text{in each instance. then we checked whether the distribution is changed or not.}$

In [ ]:
# Add 1d

fn = 0

for i in range(10):
    print('instance:',i+1,'/10')
    sprime_instance = data.sample(frac = 0.000596, replace = True, random_state = 42)
    sprime_matrix = sprime_instance.to_numpy()

    k = np.random.choice([0,1,2,3,4])
    for j in range(len_sprime):
        sprime_matrix[j,k] = np.random.normal(loc=0.0, scale=1.0)    # Standard Gaussian

    # Change detection

    p = 0.08
    stepsize = 0.002

    Delta = delta()
    C_max = critical_value(p,stepsize)

    if Delta < C_max:
        print('THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S ')

    else:
        fn += 1
        print('THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S ')

    print('fn:',fn)

print('False negative rate:',fn)

instance: 1 /10
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
instance: 2 /10
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
instance: 3 /10
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
instance: 4 /10
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
instance: 5 /10
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
instance: 6 /10
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
instance: 7 /10
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
instance: 8 /10
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
instance: 9 /10
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
instance: 10 /10
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0

$\text{Expected result: 0.}$

$\text{Obtained result: 0.}$

$\Large{\text{Scaling Single Dimensional Changes:}}$

$\text{S' is created by the value of the randomly-selected dimension is multiplied by two.}$

$\text{in each instance. then we checked whether the distribution is changed or not.}$

In [ ]:
#scale 1 d

fn = 0

for i in range(10):
    print('instance:',i+1,'/10')
    sprime_instance = data.sample(frac = 0.000596, replace = True, random_state = 42)
    sprime_matrix = sprime_instance.to_numpy()
    k = np.random.choice([0,1,2,3,4])

    for j in range(len_sprime):
        sprime_matrix[j,k] = sprime_matrix[j,k]*2

    # Change detection
    p = 0.08
    stepsize = 0.002

    Delta = delta()
    C_max = critical_value(p,stepsize)
    print(Delta,C_max)
    if Delta < C_max:
        print('THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S ')

    else:
        fn += 1
        print('THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S ')

    print('fn:',fn)

print('False negative rate:',fn)

instance: 1 /10
-284.6452082405158 -195.5775291196766
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
instance: 2 /10
-338.347899114743 -195.57752911967665
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
instance: 3 /10
-316.8983095289085 -195.5775291196766
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
instance: 4 /10
-419.8194252478321 -195.57752911967665
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
instance: 5 /10
-419.8194252478321 -195.57752911967665
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
instance: 6 /10
-419.8194252478321 -195.57752911967665
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
instance: 7 /10
-284.6452082405158 -195.5775291196766
THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S 
fn: 0
instance: 8 /10
-338.3478991147

$\text{Expected result: 0.}$

$\text{Obtained result: 0.}$